In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

import pandas as pd
import mlflow

In [2]:
data = load_iris()
x = data['data']
y = data['target']

In [3]:
x[:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [4]:
y[:3]

array([0, 0, 0])

In [5]:
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.3,random_state=23,shuffle=True)

In [6]:

# Define models and their hyperparameter grids
models = [
    ("LogisticRegression", LogisticRegression(), {"C": 0.1, "solver": "liblinear"}),
    ("SupportVectorMachine", SVC(), {"C": 0.1, "kernel":  "rbf"}),
    ("DecisionTreeClassifier", DecisionTreeClassifier(), {"max_depth": 10, "criterion":"gini"})
]


In [7]:
# mlflow.set_tracking_uri("http://127.0.0.1:5000")
import dagshub
dagshub.init(repo_owner='Ankush610', repo_name='mlflow', mlflow=True)

mlflow.set_experiment("Mlflow_Test1")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\ankus\anaconda3\envs\mlflow\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=fca51bd7-8ee1-4391-bf48-2dd031afe272&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=540c45bb3806861ae5f31f5115c02f80189cb9b924e78640f2211e640bb64e54




Accessing as Ankush610

Initialized MLflow to track repo "Ankush610/mlflow"

Repository Ankush610/mlflow initialized!

2024/12/17 23:37:53 INFO mlflow.tracking.fluent: Experiment with name 'Mlflow_Test1' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/e47e0e61719a401ca56d5d93eccf0173', creation_time=1734458873672, experiment_id='0', last_update_time=1734458873672, lifecycle_stage='active', name='Mlflow_Test1', tags={}>

In [8]:
for name,model,params in models:
    model.set_params(**params)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    class_report = classification_report(y_test,y_pred,output_dict=True)

    accuracy = class_report['accuracy']
    precision = class_report['weighted avg']['precision']
    recall = class_report['weighted avg']['recall']
    support = class_report['weighted avg']['support']

    with mlflow.start_run(run_name=name):

        mlflow.log_params(params)
        mlflow.log_param('model_name',name)
        mlflow.log_metric('accuracy',accuracy)
        mlflow.log_metric('precision',precision)
        mlflow.log_metric('recall',recall)
        mlflow.log_metric('support',support)

        mlflow.sklearn.log_model(model,'model')

2024/12/17 23:38:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LogisticRegression at: https://dagshub.com/Ankush610/mlflow.mlflow/#/experiments/0/runs/1d1356a324f940a08b4bd8f2c8c690ab
🧪 View experiment at: https://dagshub.com/Ankush610/mlflow.mlflow/#/experiments/0


2024/12/17 23:38:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run SupportVectorMachine at: https://dagshub.com/Ankush610/mlflow.mlflow/#/experiments/0/runs/ed0f4ec2280c4fe698e5fa3bbb76e32f
🧪 View experiment at: https://dagshub.com/Ankush610/mlflow.mlflow/#/experiments/0


2024/12/17 23:38:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run DecisionTreeClassifier at: https://dagshub.com/Ankush610/mlflow.mlflow/#/experiments/0/runs/348be863b2864443b5677f523c552f46
🧪 View experiment at: https://dagshub.com/Ankush610/mlflow.mlflow/#/experiments/0


In [9]:
result = mlflow.register_model('runs:/348be863b2864443b5677f523c552f46/model','DecisionTreeClassifier')

Successfully registered model 'DecisionTreeClassifier'.
2024/12/17 23:39:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: DecisionTreeClassifier, version 1
Created version '1' of model 'DecisionTreeClassifier'.


In [10]:
model = mlflow.sklearn.load_model('models:/DecisionTreeClassifier@challenger')
model

DecisionTreeClassifier(max_depth=10)